<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/ask_5_pubmed_bigtweet_mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'
100  9115  100  9115    0     0  32670      0 --:--:-- --:--:-- --:--:-- 32670
curl: (23) Failed writing body (289 != 1212)


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heel pain
2,5,increase my heart rate
3,5,tachycardia
4,8,severe arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-5.test.csv",header=None)
test.head()

,0,1
0,0,fear
1,0,scared to death
2,2,heel pain
3,2,heels hurt
4,2,pain in my right heel


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'rt',
 'sodium',
 'by',
 'that',
 'as',
 'at',
 'patients',
 'or',
 'this',
 'from',
 'drug',
 'you',
 'mg',
 'i',
 'group',
 'be',
 'disease',
 'study',
 'are',
 'treatment',
 'pain',
 'not',
 'after',
 'an',
 'it',
 'amp',
 'p',
 'have',
 'release',
 'gluten',
 'we',
 'than',
 'effect',
 'free',
 'all',
 'new',
 'no',
 'compared',
 'more',
 'using',
 'your',
 'effects',
 'groups',
 'both',
 'two',
 'our',
 'its',
 'significantly',
 'inflammatory',
 'my',
 'drugs',
 'time',
 'about',
 'between',
 'h',
 'these',
 'but',
 'day',
 'out',
 'use',
 'significant',
 'voltaren',
 'anti',
 'can',
 'which',
 'used',
 'has',
 'potassium',
 'dose',
 'placebo',
 'results',
 'also',
 'one',
 'what',
 'glutenfree',
 'do',
 's',
 'nt',
 'ds',
 'efficacy',
 'may',
 'celiac',
 'days',
 'there',
 'treated',
 '

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.225496,4.637650,0.249061,16:15


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.470651,4.286124,0.282636,16:13


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.160081,4.245474,0.286905,16:13


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.115546,4.213330,0.289881,16:06


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.723090,3.546983,0.407407,01:09


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.795845,2.999824,0.472542,01:24


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.384000,2.751115,0.528736,03:33


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.153945,2.550297,0.569604,03:18
1,1.931498,2.466613,0.573435,03:32


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.769171,2.320487,0.593870,03:30
1,1.630749,2.267158,0.600255,03:17


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.582570,2.186727,0.613027,03:25
1,1.477709,2.157645,0.624521,03:27


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.390216,2.173458,0.636015,03:18
1,1.272542,2.094143,0.636015,03:27


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.283556,2.064091,0.636015,03:13
1,1.193460,2.019906,0.643678,03:26


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.177027,2.025417,0.642401,03:19
1,1.165708,2.007884,0.652618,03:30
2,1.048078,1.982260,0.646232,03:17
3,1.002787,1.979736,0.655172,03:18


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.991294,2.021709,0.669221,03:19
1,1.082211,2.062564,0.650064,03:35
2,1.001513,2.001762,0.661558,03:34
3,0.948534,1.971539,0.664112,03:28


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.896496,1.992413,0.656450,03:25
1,0.973519,1.965876,0.662835,03:21
2,0.909662,1.973387,0.671775,03:14
3,0.879350,1.955661,0.669221,03:15


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.892835,2.032667,0.671775,03:33
1,0.908898,2.119913,0.671775,03:32
2,0.889929,2.069597,0.662835,03:33
3,0.809924,1.977696,0.666667,03:30


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.837048,2.073970,0.653895,03:31
1,0.874612,2.047682,0.664112,03:15
2,0.819138,1.996454,0.661558,03:18
3,0.839204,2.065536,0.660281,03:14


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.847661,2.107483,0.666667,03:22
1,0.863522,2.071063,0.666667,03:20
2,0.812753,2.068806,0.669221,03:19
3,0.778441,2.027361,0.671775,03:33


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.799374,2.068253,0.670498,03:32
1,0.872164,2.133042,0.656450,03:10
2,0.834839,2.142013,0.665390,03:19
3,0.768002,2.060053,0.669221,03:36


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.792431,2.145216,0.671775,03:16
1,0.817436,2.019876,0.670498,03:21
2,0.792637,2.128676,0.673052,03:11
3,0.758831,2.113595,0.666667,03:13


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.792954,2.087447,0.666667,03:31
1,0.852988,2.121247,0.665390,03:16
2,0.792632,2.030438,0.666667,03:19
3,0.744135,2.078598,0.670498,03:13


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.761188,2.140102,0.666667,03:27
1,0.772365,2.150262,0.665390,03:32


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.799020,2.144800,0.659004,03:19
1,0.751542,2.122224,0.660281,03:34


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [46]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [47]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[0, 532, 37, 182, 702, 269, 269, 269, 933, 16, 16, 16, 819, 27, 27, 27, 783, 197, 76, 197, 37, 37, 37, 37, 37, 741, 38, 184, 49, 987, 49, 49, 49, 49, 53, 785, 785, 265, 933, 817, 817, 71, 71, 71, 321, 343, 209, 864, 37, 741, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 817, 80, 80, 788, 455, 733, 384, 92, 94, 431, 431, 94, 94, 94, 409, 96, 764, 49, 98, 100, 100, 100, 100, 100, 100, 100, 100, 358, 103, 817, 712, 384, 107, 108, 108, 108, 111, 111, 111, 111, 111, 111, 111, 111, 111, 817, 119, 122, 515, 41, 41, 125, 455, 125, 125, 125, 125, 125, 125, 135, 817, 439, 137, 344, 449, 148, 817, 157, 865, 557, 4, 166, 166, 166, 171, 210, 172, 289, 289, 172, 864, 864, 864, 864, 864, 427, 817, 117, 245, 817, 165, 687, 687, 182, 190, 183, 183, 183, 183, 183, 183, 183, 183, 817, 187, 187, 190, 190, 462, 172, 700, 190, 190, 190, 190, 190, 190, 182, 462, 817, 20, 197, 197, 197, 197, 197, 202, 202, 20

In [48]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

486
0.5612009237875288
